## HERE Isolines API

works  with https://developer.here.com/documentation/routing/dev_guide/topics/resource-calculate-isoline.html

- isohrones are requested for distance of 1000 meters, 
- start points are schools, dowloaded with `osmnx`, 
- mode is pedestrian, 
- resolution is maximized.

In [41]:
import geopandas as gpd
import pandas as pd
import requests

from shapely.geometry import Point, LineString, Polygon
from shapely.geometry import box
from shapely import wkt

import matplotlib.pyplot as plt
import numpy as np

# maps libraries
import folium
from pyproj import CRS
import contextily as ctx
import osmnx as ox

import warnings
warnings.filterwarnings('ignore')

### importing data about schools

In [42]:
# reading api keys
api_keys = pd.read_excel('../api_keys.xlsx')
api_keys.set_index('key_name', inplace=True)

# link for mapbox map as a underlay for folium
map_url = api_keys.loc['mapbox_map']['key']

# HERE Rest api key
here_api_key = api_keys.loc['here_rest_api']['key']

In [43]:
# long place name for geocoding in OSM
place_name = 'городское поселение Альметьевск'

In [44]:
# espg for UTM CRS
# can be found via https://www.latlong.net/lat-long-utm.html and https://spatialreference.org/
espg = 32639

In [45]:
# Get the schools inside place

# tags for schools
tags = {'amenity': 'school'}

schools = ox.geometries_from_place(place_name, tags)
# Check the result
print("Retrieved", len(schools), "objects")

# reproject schools to UTM CRS
schools = schools.to_crs(epsg=espg)

Retrieved 30 objects


In [46]:
schools.head(2)

C:\Users\UX306\anaconda3\envs\gis\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,unique_id,osmid,element_type,amenity,name,geometry,barrier,nodes,addr:housenumber,addr:street,building,emergency,ways,type
0,node/971859355,971859355,node,school,ДЮСШ по шахматам,POINT (583728.507 6084657.627),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node/2265502124,2265502124,node,school,Детская музыкальная школа №1,POINT (582351.422 6084654.986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# centroids from schools in CRS 4326
centroids = schools.centroid.to_crs(epsg=4326)

#GDF out of geoseries
centroids = gpd.GeoDataFrame(geometry=gpd.GeoSeries(centroids))

#columns with lon lat
centroids['lon'] = centroids['geometry'].map(lambda p: p.x)
centroids['lat'] = centroids['geometry'].map(lambda p: p.y)

C:\Users\UX306\anaconda3\envs\gis\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
df_sample = centroids

C:\Users\UX306\anaconda3\envs\gis\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### request to Here API

In [49]:
# here API url
URL = 'https://isoline.route.ls.hereapi.com/routing/7.2/calculateisoline.json'

In [50]:
#distance in meters
distance = 1000

In [51]:
# returns isohrone polygons in WKT format

def isoline_polygon_wkt(lat, lon):
    #start point of isoline
    start_point = f"geo!{lat:.6f},{lon:.6f}"
    # request params
    params = { 
        'apiKey': here_api_key,
        'start': start_point,
        'range': 1000,
        'rangetype': 'distance',
        'mode': 'fastest;pedestrian',
        'departure': 'now',
        'singlecomponent': 'true',
        'resolution': 10,
        'maxpoints': 300,
            }
    # response
    response = requests.get(URL, params=params)
    response_json = response.json()
    
    try:
        # resulting polygon
        polygon = response_json['response']['isoline'][0]['component'][0]['shape']

        # creating dataframe with lon lat columns out of result
        poly_df = pd.DataFrame(polygon)
        coordinates = poly_df[0].str.split(",", expand = True)
        coordinates.columns = ['lat', 'lon']

        # lists of coordinates
        coordinates['coord'] = coordinates['lon'].str.cat(coordinates['lat'],sep=" ")
        coord = (coordinates['coord'].to_list())

        coord_str = 'POLYGON ((' + ','.join(coord) + '))'
    
        return coord_str
    
    except Exception as e:
        print("for point", start_point)
        print("result is", response_json)
        print("which raises", e)
        return ""

In [52]:
df_sample['isoline_wkt'] = df_sample.apply(lambda df_sample: isoline_polygon_wkt
                                        (df_sample['lat'],df_sample['lon']), axis=1)
# drop point geometry
df_sample = df_sample.drop('geometry',axis=1).rename({'isoline_wkt':'geometry'}, axis=1)

In [53]:
# read wkt format

def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None
    
df_sample['geometry'] = df_sample['geometry'].apply(wkt_loads)
df_sample = df_sample.dropna(subset=['geometry'])

#create geodataframe
isolines = gpd.GeoDataFrame(df_sample, geometry='geometry')
isolines.head()

,lon,lat,geometry
0,52.305757,54.901917,"POLYGON ((52.29432 54.90074, 52.29475 54.90091..."
1,52.284286,54.902122,"POLYGON ((52.28127 54.89868, 52.28170 54.89885..."
2,52.280382,54.908931,"POLYGON ((52.27098 54.90761, 52.27140 54.90778..."
3,52.282748,54.908787,"POLYGON ((52.27098 54.90761, 52.27140 54.90778..."
4,52.265641,54.904700,"POLYGON ((52.25518 54.90967, 52.25561 54.90950..."


### folium map with results

In [54]:
m = folium.Map(location=[54.903436, 52.295439],  zoom_start = 13, control_scale=True)

#Convert cells to GeoJSON, select CRS 3857, add to map
folium.GeoJson(
   isolines.to_crs(epsg=3857),
    name='isolines'
).add_to(m)

folium.GeoJson(
   centroids.to_crs(epsg=3857),
    name='schools'
).add_to(m)

m

C:\Users\UX306\anaconda3\envs\gis\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
# write the result in needed UTM CRS to Shapefile
isolines.to_crs(epsg=espg).to_file('./output/isolines.shp')